In [1]:
import os
import joblib
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from google.cloud import bigquery, storage
from xgboost import XGBRegressor

os.chdir('/home/jupyter/projects_data/sales_budget')

project_id = 'valid-heuristic-369117'

df = pd.read_csv('Advertising.csv', index_col=[0])
display(df.shape, df.head())

(200, 4)

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [2]:
X = df.copy()
y = X.pop('Sales')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = int(0.25*X.shape[0]))
display(X_train.shape, X_test.shape, y_train.shape, X_train.head())

(150, 3)

(50, 3)

(150,)

,TV,Radio,Newspaper
35,95.7,1.4,7.4
72,109.8,14.3,31.7
173,19.6,20.1,17.0
145,96.2,14.8,38.9
181,156.6,2.6,8.3


In [3]:
lm = LinearRegression()
lm.fit(X_train, y_train)
lm.predict(X_test)

array([ 6.31993695,  9.94336236, 12.15814505, 21.18928245, 17.28629955,
       18.25203448, 18.19640551,  9.95243515, 15.09120348, 20.73819586,
       20.34545299, 12.25756158, 16.48441597, 17.38818168, 14.52544995,
       15.48044595, 14.08803788, 18.92037569, 20.63734031, 24.14352823,
        5.47380599, 19.32984346,  4.72826597, 14.29492732, 12.6499803 ,
       15.97590893, 10.73759922, 18.73580674, 23.36613185, 15.56557916,
       13.69852487, 16.87902173, 18.41728609, 12.29666963, 14.50499392,
       15.98959144, 16.57790548, 14.40558085,  6.22819974, 19.22499512,
       11.80732031, 14.28060873, 15.39676294, 18.49640029, 13.86401812,
       17.33644639,  6.95635873, 13.99689334, 11.64089067,  6.63663245])

In [4]:
xgbm = XGBRegressor(eta=0.1, max_depth=5, subsample=0.6)
xgbm.fit(X_train, y_train)
xgbm.predict(X_test)

array([ 7.9519887, 11.126699 ,  9.121286 , 22.554003 , 15.330488 ,
       15.39732  , 18.89791  ,  9.85052  , 13.028841 , 22.763586 ,
       20.957296 , 10.907264 , 16.145878 , 12.976828 , 14.861608 ,
       13.28401  , 13.810883 , 16.259327 , 22.607218 , 25.491957 ,
        4.450262 , 20.01592  ,  6.5037932, 14.299456 , 12.419514 ,
       12.059183 ,  8.423335 , 17.474552 , 24.433163 , 15.99235  ,
       12.30582  , 18.24088  , 19.829275 ,  7.3567443, 14.473753 ,
       12.274988 , 16.06165  , 13.783503 ,  7.238706 , 19.804949 ,
       11.746248 , 15.317836 , 14.792433 , 19.700937 , 14.698485 ,
       17.518156 ,  7.4104123, 11.256359 , 11.958614 ,  5.8302426],
      dtype=float32)

In [5]:
print('In sample')
print(r2_score(y_train, lm.predict(X_train)), r2_score(y_train, xgbm.predict(X_train)))
print('Out of sample')
print(r2_score(y_test, lm.predict(X_test)), r2_score(y_test, xgbm.predict(X_test)))

In sample
0.9027396078925684 0.9993679804432034
Out of sample
0.8736482557069646 0.9868609494525639


In [6]:
os.chdir('/home/jupyter/project_repos/pg_sales_budget/sales-app')
artifact_filename_lm = 'lr_model.pkl'
joblib.dump(lm, artifact_filename_lm)

artifact_filename_xgb = 'xgb_model.json'
xgbm.save_model(artifact_filename_xgb)

In [7]:
model_bucket = 'gs://pmykola-projectsgcp-artifacts/sales-budget'
storage_path = os.path.join(model_bucket, artifact_filename_lm)
blob = storage.blob.Blob.from_string(storage_path, client=storage.Client(project=project_id))
blob.upload_from_filename(os.getcwd()+'/'+artifact_filename_lm)

In [8]:
model_bucket = 'gs://pmykola-projectsgcp-artifacts/sales-budget'
storage_path = os.path.join(model_bucket, artifact_filename_xgb)
blob = storage.blob.Blob.from_string(storage_path, client=storage.Client(project=project_id))
blob.upload_from_filename(os.getcwd()+'/'+artifact_filename_xgb)

In [9]:
test_data = [159.1, 60.2, 90]
test_data = np.array(test_data).astype(np.float)
test_data = test_data.reshape(1,-1)
test_data

filePath = 'lr_model.pkl'
file = open(filePath, "rb")
trained_model = joblib.load(file)
prediction = trained_model.predict(test_data)
print('lm', prediction)

trained_model = XGBRegressor()
trained_model.load_model(artifact_filename_xgb)
prediction = trained_model.predict(test_data)
print('xgb', prediction)


lm [21.25726952]
xgb [20.073584]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


In [10]:
import xgboost
xgboost.__version__

'1.6.2'